In [1]:
# build poetry's book latex source code from .md file repository

In [2]:
# global definitions

REPO_PATH = "../.."
POEM_PATH_SUBSTRING = "/songs/en/"

DEBUG_ADD_PATH = False

TEX_OUTPUT_FILENAME = "/tmp/output.tex"
PDF_OUTPUT_FOLDER = "/tmp"

TEX_BLANK = r"""
\begin{titlepage}
  \centering
  \vfill
\end{titlepage}
"""

TEX_PREAMBLE = r"""
% compile with lualatex
\documentclass[12pt]{book}
\usepackage{calc,emptypage,fontspec,graphicx,microtype,verse,xcolor}

% a free version of Goudy’s Deepdene from www.theleagueofmoveabletype.com:
\setmainfont{Linden Hill}
\linespread{1.10344}

\setlength{\topskip}{\baselineskip}
\usepackage[a5paper,hcentering,heightrounded,includeheadfoot]{geometry}

\definecolor{myBGcolor}{HTML}{FFFFFF}
\definecolor{myTextcolor}{HTML}{000000}
\pagecolor{myBGcolor}
\color{myTextcolor}

% I’ve reduced the default left margin for verse to prevent lines from
% breaking:
\setlength{\leftmargini}{1em}

% One of many customizations made possible by the verse package:
\renewcommand{\poemtitlefont}{\normalfont\large\centering}

% This is said to help Kindles, but I don’t have one for testing:
\usepackage[a5,noinfo,center,frame,color=myBGcolor!99]{crop}
\crop[frame]

% toc - if you make one and if your font has anything resembling a bold
% weight, LaTeX will try to use it in your TOC.  Bold and poetry don’t
% mix; here’s a quick, brute-force method to get rid of all bold type:
\let\bfseries\mdseries

% Get rid of blank pages if this is to be read on screen:
%\let\cleardoublepage\clearpage

\usepackage{bookmark}
\hypersetup{hidelinks,pdftitle={The Lyrics},pdfauthor={Dror Kessler}}
\pagestyle{plain}
\begin{document}
\frontmatter

\begin{titlepage}
  \centering
  \vspace*{.1\textheight}

  \resizebox{.9\textwidth}{!}{\addfontfeatures{Kerning=Uppercase}The Lyrics}

  \vspace*{.1\textheight}

  {\huge Dror Kessler\par}

  \vspace*{.1\textheight}

  {xxxTIMESTAMPxxx\par}

\vfill

%  \includegraphics[scale=.36]{drorkessler}
\end{titlepage}

% blank page before contents
\begin{titlepage}
  \centering
  \vfill
\end{titlepage}

\tableofcontents

\mainmatter
"""

In [3]:
# produce list of .md files of individual poems
import os

def poem_files():
    result = []
    for path, subdirs, files in os.walk(REPO_PATH):
        for name in files:
            fullpath = os.path.join(path, name)
            if POEM_PATH_SUBSTRING in fullpath:
                result.append(fullpath)
                
    return result

#poem_files()

In [4]:
# parse a poem file

def parse_poem_file(filename):
    poem = {"title": "", "blocks":[], "path":filename, "instrumental":False}
    block = []
    
    with open(filename, 'r', encoding='UTF-8') as file:
        while line := file.readline():
            line = line.strip();
            
            if line.startswith("#"):
                poem["title"] = line[2:]
            elif line.startswith("---"):
                break
            else:
                if line.endswith('\\'):
                    line = line[:-1].strip()
                if len(line):                    
                    block.append(line)
                    if "(instrumental)" in line.lower():
                        poem["instrumental"] = True
                else:
                    if len(block):
                        poem["blocks"].append(block)
                    block = []

                
    if len(block):
        poem["blocks"].append(block)

    return poem
            
#parse_poem_file(poem_files()[0])

In [5]:
# build tex for a single poem

def poem_tex(poem):
    tex = []
    tex.append("\\poemtitle{" + poem["title"] + "}")
    
    tex.append("\\begin{verse}")

    if DEBUG_ADD_PATH:
        tex.append(poem["path"])
        tex.append("")

    for block in poem["blocks"]:
        for line in block[:-1]:
            tex.append(line + "\\\\*")
        tex.append(block[-1])
        tex.append("")
    
    tex.append("\\end{verse}")
    
    return "\n".join(tex).replace("&", "\\&")

#print(poem_tex(parse_poem_file(poem_files()[0])))

In [6]:
# create output tex
from datetime import date
from lorem_text import lorem
today = date.today()

with open(TEX_OUTPUT_FILENAME, 'w', encoding='UTF-8') as file:
    
    # write preamble
    preamble = TEX_PREAMBLE.replace("xxxTIMESTAMPxxx", today.strftime("%B, %Y"))
    
    file.write(preamble)
    
    # get poems
    poems = [parse_poem_file(f) for f in poem_files()]
    
    # sort alphabetically
    poems = sorted(poems, key=lambda p: p["title"])
    print("%s poems" % len(poems))
    
    # establish range
    #hunt_inst = [0, 0, 0] succ
    hunt_inst = []
    p_from = 0
    p_to = len(poems)
    for inst in hunt_inst:
        p_size = (p_to - p_from) / 2
        if not inst:
            p_to -= p_size
        else:
            p_from += p_size
    p_from = int(round(p_from))
    p_to = int(round(p_to))
    
    #p_to -= 1
    
    # write poems
    print("writing poems [%d-%d)\n" % (p_from, p_to))
    for poem in poems[p_from:p_to]:
        if not poem["instrumental"]:
            #print("%d: %s" % (p_from, poem["title"]))
            p_from += 1
            
            tex = poem_tex(poem)
            file.write(tex)
            file.write("\n")
            file.write("\\clearpage\n")
            file.write("\n")
        
    # add blank pages
    blanks = 0
    print("adding %d blanks" % blanks)
    for _ in range(blanks):
        poem = {"title": "X", "blocks":[], "path":"", "instrumental":False}
        for _ in range(2):
            block = []
            for _ in range(1):
                block.append(lorem.sentence())
            poem["blocks"].append(block)
        tex = poem_tex(poem)
        file.write(tex)
        file.write("\n")
        file.write("\\clearpage\n")
        file.write("\n")
        
    # close file
    file.write("\\end{document}\n");


148 poems
writing poems [0-148)

adding 0 blanks


In [7]:
# generate pdf
import os

cmd = "lualatex --output-directory " + PDF_OUTPUT_FOLDER + " " + TEX_OUTPUT_FILENAME
os.system(cmd)


This is LuaHBTeX, Version 1.16.0 (TeX Live 2023) 
 restricted system commands enabled.
(/tmp/output.tex
LaTeX2e <2022-11-01> patch level 1
 L3 programming layer <2023-02-22>
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/book.cls
Document Class: book 2022/07/02 v1.4n Standard LaTeX document class
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/bk12.clo))
(/usr/local/texlive/2023/texmf-dist/tex/latex/tools/calc.sty)
(/usr/local/texlive/2023/texmf-dist/tex/latex/emptypage/emptypage.sty)
(/usr/local/texlive/2023/texmf-dist/tex/latex/fontspec/fontspec.sty
(/usr/local/texlive/2023/texmf-dist/tex/latex/l3packages/xparse/xparse.sty
(/usr/local/texlive/2023/texmf-dist/tex/latex/l3kernel/expl3.sty
(/usr/local/texlive/2023/texmf-dist/tex/latex/l3backend/l3backend-luatex.def)))
 (/usr/local/texlive/2023/texmf-dist/tex/latex/fontspec/fontspec-luatex.sty
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/fontenc.sty)
(/usr/local/texlive/2023/texmf-dist/tex/latex/fontspec/fontspec.cfg)))
(/us

0